# Chapter 1. Simple tasks
## 2.0. Tree data structure

In [1]:
class Node:
    def __init__(self, data):
        self.left = None
        self.right = None
        self.data = data
    
    def insert(self, data):
        # compare new value with current
        if self.data:
            if data < self.data:
                if self.left == None:
                    self.left = Node(data)
                else:
                    self.left.insert(data)
            elif data > self.data:
                if self.right == None:
                    self.right = Node(data)
                else:
                    self.right.insert(data)
        else:
            self.data = data
    
    def print_tree(self):
        if self.left:
            self.left.print_tree()
        print(self.data)
        if self.right:
            self.right.print_tree()
    
    def inorder_traversal(self, root):
        res = []
        if root:
            res = root.inorder_traversal(root.left)
            res.append(root.data)
            res += root.inorder_traversal(root.right)
        return res

In [2]:
tree = Node(25)
tree.insert(12)
tree.insert(32)
tree.insert(3)
tree.insert(83)

In [3]:
tree.inorder_traversal(tree)

[3, 12, 25, 32, 83]

In [4]:
tree.print_tree()

3
12
25
32
83


## 2.1. DNA search
DNA search

In [5]:
from enum import IntEnum
from typing import Tuple, List

Nucleotide: IntEnum = IntEnum('Nucleotide', ('A', 'C', 'G', 'T'))
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene = List[Codon]

def string_to_gene(s: str) -> Gene:
    gene: Gene = []
    for i in range(0, len(s), 3):
        if (i + 2) >= len(s): # Do not move over the range
            return gene
        codon: Codon = (Nucleotide[s[i]], Nucleotide[s[i+1]], Nucleotide[s[i+2]])
        gene.append(codon)
    return gene

In [6]:
str_gene = "ATGCGTATGCGTTTTGATATCTCAGATTTAATGCGTATGCGTTTTGATATCTCAGATTTAATGCGTATGCGTTTTGATATCTCAGATTTAT"
str_gene = "ATGCGTTTTATGCGTGATTTTATGCGTGATTTT"
test_gene: Gene = string_to_gene(str_gene)

__Linear search__

In [7]:
def linear_contains(gene: Gene, key_codon: Codon) -> Tuple[bool, int]:
    counter = 0
    for codon in gene:
        counter += 1
        if codon == key_codon:
            return True, counter
    return False, counter

In [8]:
acg: Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat: Codon = (Nucleotide.G, Nucleotide.A, Nucleotide.T)

print(linear_contains(test_gene, acg))

(False, 11)


In [9]:
print(linear_contains(test_gene, gat))

(True, 6)


__Binary search__

In [10]:
def binary_contains(gene: Gene, key_codon: Codon) -> Tuple[bool, int]:
    low: int = 0
    high: int = len(gene) - 1
    counter: int = 0
    while low <= high:
        counter += 1
        mid: int = (low + high) // 2
        if gene[mid] < key_codon:
            
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True, counter
    return False, counter

In [11]:
print(binary_contains(test_gene, acg))

(False, 3)


In [12]:
print(binary_contains(test_gene, gat))

(True, 1)


In [13]:
def binary_recursive_contains(gene: Gene, key_codon: Codon, low: int, high: int) -> bool:
    
    mid: int = (low + high) // 2
    if low > high:
        print("exceeded")
        return False
    if gene[mid] < key_codon:
        print("Less")
        low = mid + 1
        binary_recursive_contains(gene, key_codon, low, high)
    elif gene[mid] > key_codon:
        print("More")
        high = mid - 1
        binary_recursive_contains(gene, key_codon, low, high)
    elif gene[mid] == key_codon:
        print("Equal")
        return True

In [14]:
print(binary_recursive_contains(test_gene, acg, 0, len(test_gene) - 1))

More
More
More
exceeded
None


__Parametrized example__

In [17]:
from __future__ import annotations
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from typing_extensions import Protocol
from heapq import heappush, heappop

In [18]:
T = TypeVar('T') # Can be anything
# if specified: T = TypeVar('T', str, bytes) it should be str or bytes

def linear_contains(iterable: Iterable[T], key: T) -> bool:
    for item in iterable:
        if item == key:
            return True
    return False

In [19]:
C = TypeVar('C', bound="Comparable")

class Comparable(Protocol):
    def __eq__(self, other: Any) -> bool:
        pass
    
    def __lt__(self, other: Any) -> bool:
        pass
    
    def __gt__(self, other: Any) -> bool:
        return (not self < other) and self != other
    
    def __le__(self, other: Any) -> bool:
        return self < other or self == other
    
    def __ge__(self, other: Any) -> bool:
        return not self < other
    
    def binary_contains(sequence: Sequence[C], key: C) -> bool:
        low: int = 0
        high: int = len(sequence) - 1
        while low <= high:
            mid: int = (low + high) / 2
            if sequence[mid] < key:
                low = mid + 1
            elif sequence[mid] > key:
                high = mid - 1
            else:
                return True
        return False

In [20]:
# Test case 1
print(linear_contains([1, 5, 15, 15, 15, 20], 5))

True


In [21]:
# Test case 2
print(binary_contains(["a", "d", "e", "f", "z"], "f"))

(True, 2)


In [22]:
# Test case 3
print(binary_contains(["john", "mark", "ronald", "sarah"], "sheila"))

(False, 3)


## 2.2. Labyrinth

__Building a maze__

In [23]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt

In [69]:
class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    GOAL = "G"
    START = "S"
    PATH = "*"
    
class MazeLocation(NamedTuple):
    row: int
    column: int

class Maze:
    
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2,
                start: MazeLocation = MazeLocation(0,0), goal: MazeLocation = MazeLocation(9,9)) -> None:
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
            
        # Fill maze with empty blocks
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        # Randomly fill maze with blocked blocks
        self._randomly_fill(rows, columns, sparseness)
        # Fill start and end positions
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
        
        
    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
          
        
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    
    
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        """
        Checking free spaces in upper, lower, right, and left locations
        around current maze locations
        """
        locations: List[MazeLocation] = []
        
        if (ml.row + 1 < self._rows # Check if there is space
           and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED): # Check if there is free path
            locations.append(MazeLocation(ml.row + 1, ml.column))
        
        if (ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED):
            locations.append(MazeLocation(ml.row - 1, ml.column))
        
        if (ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED):
            locations.append(MazeLocation(ml.row, ml.column + 1))
        
        if (ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED):
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    
    
    def mark(self, path: List[MazeLocation]):
        """
        Show the path
        """
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
        
    
    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
     
        
    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output

In [70]:
maze: Maze = Maze(sparseness = 0.2)
print(maze)

S   X    X
          
 X  XX   X
   X    X 
    X     
  X       
         X
      X   
 X X XX XX
  X    X G



__Depth-first search__  
Implement stack, first

In [71]:
class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
    
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
    
    def pop(self) -> T:
        return self._container.pop()
    
    def __repr__(self) -> str:
        return repr(self._container)

In [72]:
my_stack = Stack()
my_stack.push(1)
my_stack.push(5)
my_stack.push(10)
print(f"After pushing: {my_stack}")

val = my_stack.pop()
print(f"Popped: {val}, got: {my_stack}")

print(f"Is empty: {my_stack.empty}")

After pushing: [1, 5, 10]
Popped: 10, got: [1, 5]
Is empty: False


Implement Node as a wrapup around state

In [73]:
class Node(Generic[T]):
    def __init__(self, state: T, parent: Optional[Node], cost: float = 0.0, heuristic: float = 0.0) -> None:
        self.state: T = state
        self.parent: Optional[Node] = parent
        self.cost: float = cost
        self.heuristic: float = heuristic
    
    def __lt__(self, other: Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

DFS - check two data structures: stack of reviewed states and set of explored states

In [74]:
def dfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    # Frontier - next steps
    frontier: Stack[Node[T]] = Stack()
    # Push the first slot
    frontier.push(Node(initial, None))
    # Places where we have been
    explored: Set[T] = {initial}
        
    # Check all the states, while they are available
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        
        # If found, finish
        if goal_test(current_state):
            return current_node
    
        # Check where to go next
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None # If there is no solution

Restore path

In [75]:
def node_to_path(node: Node[T]) -> List[T]:
    path: List[T] = [node.state]
    # From the end to the begining
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path

__Test DFS__

In [78]:
m: Maze = Maze(sparseness=0.3)
print(m)

S     X X 
X X X  XXX
  XX  X  X
 X  X XX  
    X     
  X  X X  
  X       
     X X  
          
  XXX XXXG



In [79]:
solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.successors)
if solution1 is None:
    print("No solution found using depth-first search")
else:
    path1: List[MazeLocation] = node_to_path(solution1)
    m.mark(path1)
    print(m)
    m.clear(path1)

S*****X X 
X X X* XXX
  XX *X  X
 X  X*XX  
****X*****
* X* X X *
**X*******
 **  X X  
  ********
  XXX XXXG



__Breadth-first search__  
Always finds the shortest path

Implement queue, at first

In [80]:
class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()
        
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
        
    def pop(self) -> T:
        return self._container.popleft()
    
    def __repr__(self) -> str:
        return repr(self._container)        

Implement BFS

In [84]:
def bfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    frontier: Queue[Node[T]] = Queue()
    frontier.push(Node(initial, None))
    explored: Set[T] = {initial}
        
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        if goal_test(current_state):
            return current_node
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None

Test BFS

In [85]:
solution2: Optional[Node[MazeLocation]] = bfs(m.start, m.goal_test, m.successors)

if solution2 is None:
    print("No solution found using breadth-first search!")
else:
    path2: List[MazeLocation] = node_to_path(solution2)
    m.mark(path2)
    print(m)
    m.clear(path2)

S*****X X 
X X X* XXX
  XX *X  X
 X  X*XX  
    X**   
  X  X*X  
  X   *   
     X*X  
      ****
  XXX XXXG



__A* search__  
Focus on the paths that are more likely to lead to the goal

Priority Queue

In [87]:
class PriorityQueue(Generic[T]):
    """
    Priority is taken as cost value for element (__lt__ in Node class)
    """
    def __init__(self) -> None:
        self._container: List[T] = []
        
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        heappush(self._container, item)
        
    def pop(self) -> T:
        return heappop(self._container)
    
    def __repr__(self) -> str:
        return repr(self._container)

Heuristics: Euclidian distance

In [88]:
def euclidian_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    # Goal is fixed in outer function
    def distance(ml: MazeLocation) -> float:
        xdist: int = ml.column - goal.column
        ydist: int = ml.row - goal.row
        return sqrt((xdist * xdist) + (ydist * ydist))
    return distance

Heuristics: Manhattan distance

In [89]:
def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = abs(ml.column - goal.column)
        ydist: int = abs(ml.row - goal.row)
        return (xdist + ydist)
    return distance

A*-algorithm implementation

In [93]:
def astar(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]], 
          heuristic: Callable[[T], float]) -> Optional[Node[T]]:
    frontier: PriorityQueue[Node[T]] = PriorityQueue()
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    explored: Dict[T, float] = {initial: 0.0}
        
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
            
        if goal_test(current_state):
            return current_node
        
        for child in successors(current_state):
            new_cost: float = current_node.cost + 1 # Applicable only for simple cases like grid
            
            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    return None

Testing A* search

In [94]:
distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)
solution3: Optional[Node[MazeLocation]] = astar(m.start, m.goal_test, m.successors, distance)

if solution3 is None:
    print("No solution found using A*!")
else:
    path3: List[MazeLocation] = node_to_path(solution3)
    m.mark(path3)
    print(m)

S*****X X 
X X X* XXX
  XX *X  X
 X  X*XX  
    X**   
  X  X*X  
  X   *   
     X*X  
      ****
  XXX XXXG



## 2.3. Missionaries
Left bank |....| Right bank  
C1 C2 C3  |....|    
M1 M2 M3  |B...|  

In [95]:
from __future__ import annotations
from typing import List, Optional

In [111]:
MAX_NUM = 3

class MCState:
    def __init__(self, missionaries: int, cannibals: int, boat: bool) -> None:
        self.wm: int = missionaries # west coast missionaries
        self.wc: int = cannibals
        self.em: int = MAX_NUM - self.wm
        self.ec: int = MAX_NUM - self.wc
        self.boat: bool = boat
            
    def goal_test(self) -> bool:
        return self.is_legal and self.em == MAX_NUM and self.ec == MAX_NUM
    
    def successors(self) -> List[MCState]:
        sucs: List[MCState] = []
        if self.boat: # boat at west bank
            if self.wm > 1:
                sucs.append(MCState(self.wm - 2, self.wc, not self.boat))
            if self.wm > 0:
                sucs.append(MCState(self.wm - 1, self.wc, not self.boat))
            if self.wc > 1:
                sucs.append(MCState(self.wm, self.wc - 2, not self.boat))
            if self.wc > 0: 
                sucs.append(MCState(self.wm, self.wc - 1, not self.boat))
            if (self.wc > 0) and (self.wm > 0):
                sucs.append(MCState(self.wm - 1, self.wc - 1, not self.boat))
        else: # boat at east bank
            if self.em > 1:
                sucs.append(MCState(self.wm + 2, self.wc, not self.boat))
            if self.em > 0:
                sucs.append(MCState(self.wm + 1, self.wc, not self.boat))
            if self.ec > 1:
                sucs.append(MCState(self.wm, self.wc + 2, not self.boat))
            if self.ec > 0: 
                sucs.append(MCState(self.wm, self.wc + 1, not self.boat))
            if (self.ec > 0) and (self.em > 0):
                sucs.append(MCState(self.wm + 1, self.wc + 1, not self.boat))
        return [x for x in sucs if x.is_legal]
    
    @property
    def is_legal(self) -> bool:
        # Define forbidden steps
        if self.wm < self.wc and self.wm > 0:
            return False
        if self.em < self.ec and self.em > 0:
            return False
        return True
    
    def __str__(self) -> str:
        return ("On the west bank there are {} missionaries and {} cannibals. \n\
On the east bank there are {} missionaries and {} cannibals. \n\
The boat is on the {} bank.".format(self.wm, self.wc, self.em, self.ec, 
                                                   ("west" if self.boat else "east")))

In [112]:
def display_solution(path: List[MCState]):
    if len(path) == 0:
        return
    old_state: MCState = path[0]
    print(old_state)
    for current_state in path[1:]:
        if current_state.boat:
            print("{} missionaries and {} cannibals moved from the east to the west bank.\n".format(old_state.em - current_state.em, old_state.ec - current_state.ec))
        else:
            print("{} missionaries and {} cannibals moved from the west to the east bank.\n".format(old_state.wm - current_state.wm, old_state.wc - current_state.wc))
        print(current_state)
        old_state = current_state

In [113]:
state = MCState(3,3,True)
print(state)

On the west bank there are 3 missionaries and 3 cannibals. 
On the east bank there are 0 missionaries and 0 cannibals. 
The boat is on the west bank.


Solution to the problem

In [114]:
start: MCState = MCState(MAX_NUM, MAX_NUM, True)
solution: Optional[Node[MCState]] = bfs(start, MCState.goal_test, MCState.successors)

if solution is None:
    print("No solution found!")
else:
    path: List[MCState] = node_to_path(solution)
    display_solution(path)

On the west bank there are 3 missionaries and 3 cannibals. 
On the east bank there are 0 missionaries and 0 cannibals. 
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west to the east bank.

On the west bank there are 3 missionaries and 1 cannibals. 
On the east bank there are 0 missionaries and 2 cannibals. 
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east to the west bank.

On the west bank there are 3 missionaries and 2 cannibals. 
On the east bank there are 0 missionaries and 1 cannibals. 
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west to the east bank.

On the west bank there are 3 missionaries and 0 cannibals. 
On the east bank there are 0 missionaries and 3 cannibals. 
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east to the west bank.

On the west bank there are 3 missionaries and 1 cannibals. 
On the east bank there are 0 missionaries and 2 cannibals. 
